# Score: 0.1385

In [56]:
import numpy as numpy
import pandas as pd

In [57]:
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data

In [58]:
prices_full = pd.read_csv('./data/train.csv')
prices_test = pd.read_csv('./data/test.csv')
prices_full

In [59]:
prices_full.describe()

In [60]:
plt.figure(figsize=(15, 15))
sns.heatmap(prices_full.corr())

In [61]:
prices_full.info()

In [62]:
plt.figure(figsize=(10, 10))
sns.heatmap(prices_full.isna())

In [63]:
X_train = prices_full.drop(columns='SalePrice')
y_train = prices_full['SalePrice']

X_test = prices_test

### Drop Columns

In [64]:
drop_cols = ['Alley', 'PoolQC', 'MiscFeature', 'Fence']
prices_full.drop(columns=drop_cols, inplace=True)

In [72]:
num_cols, cat_cols = [], []
for col in X_train:
    if X_train[col].dtype == 'object':
        cat_cols.append(col)
    else:
        num_cols.append(col) 

In [73]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('scaler', MaxAbsScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])

In [74]:
from xgboost import XGBRegressor

In [75]:
model = XGBRegressor(n_estimators=250, learning_rate=0.03)

In [76]:
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

In [77]:
my_pipeline.fit(X_train, y_train)

In [78]:
preds = my_pipeline.predict(X_test)

In [79]:
pd.DataFrame({'Id': prices_test.Id, 'SalePrice': preds}).to_csv('./data/submission.csv', index=False)